In [1]:
import subprocess
import sys
import time

In [2]:
import ipynb.fs.full.utils as utils
import ipynb.fs.full.blockchain as blockchain_utils
import ipynb.fs.full.device as device_utils
import ipynb.fs.full.oracle as oracle

### LOAD RESOURCES

In [3]:
settings = utils.load_yaml('config/settings.yaml')

In [4]:
latest = utils.load_json('config/latest.json')

In [5]:
device_info = utils.load_yaml('config/identifier.yaml')

### CONNECT TO ETHEREUM GATEWAY

In [6]:
web3 = blockchain_utils.connect(settings)

KeyboardInterrupt: 

### EXTRACT THE WHISPER API

In [ ]:
shh = web3.geth.shh

### GENERATE A FRESH WHISPER ID

In [ ]:
whisper_id = shh.newKeyPair()

### CREATE DEVICE OUTLINE

In [ ]:
oracle = device_utils.create(device_info)

### SERIALIZE NECESSARY MANAGER CONTRACTS

In [ ]:
oracle_manager = blockchain_utils.contract(latest['oraclemanager'], web3, settings)

In [ ]:
task_manager = blockchain_utils.contract(latest['taskmanager'], web3, settings)

### FETCH & SERIALIZE ORACLE CONTRACT

In [ ]:
temp_contract = blockchain_utils.contract({
    'address': oracle_manager.read({
        'func': 'fetch_oracle',
        'params': oracle.hash
    }),
    'abi': latest['oracle']['abi']
}, web3, settings)

### VERIFY ORACLE CONTRACT EXISTENCE

In [ ]:
if temp_contract.address != '0x0000000000000000000000000000000000000000':
    oracle.set_contract(temp_contract)
    
else:
    log('THE ORACLE IS NOT REGISTERED, ABORTING...')
    sys.exit(0)

### GLOBAL TASK BACKLOG

In [ ]:
raw = oracle.read('details')[5]
backlog = utils.filter_backlog(raw)

### GLOBAL ACTIVE STATUS 

In [ ]:
active = oracle.read('active')

### GLOBAL DISCOVERY STATUS

In [ ]:
discoverable = oracle.read('discoverable')

### GLOBAL DISCOVERY CONFIG

In [ ]:
encoded = oracle.read('config')

In [ ]:
discovery_config = utils.decode(encoded)

### UDATE DEVICE STATUS & DETAILS

In [ ]:
def update_details(event):
    
    # FETCH GLOBAL VARS
    global active
    global discoverable
    global discovery_config
    global backlog
    
    # EXTRACT RELEVANT VALUES
    latest_active = event['args']['active']
    latest_discoverable = event['args']['discoverable']
    latest_config = decode(event['args']['config'])

    # IF ACTIVE STATUS HAS CHANGED
    if (latest_active != active):
        
        # UPDATE ACTIVE STATUS
        active = latest_active
        
        # SEND MSG
        utils.log('ACTIVE STATUS CHANGED TO: ' + str(latest_active))
        
    # IF DISCOVERABLE STATUS HAS CHANGED
    if (latest_discoverable != discoverable):
        
        # UPDATE ACTIVE STATUS
        discoverable = latest_discoverable
        
        # SEND MSG
        utils.log('DISCOVERABLE STATUS CHANGED TO: ' + str(latest_discoverable))
        
    # IF DISCOVERABLE STATUS HAS CHANGED
    if (latest_config != discovery_config):
        
        # UPDATE ACTIVE STATUS
        discovery_config = latest_config
        
        # SEND MSG
        log('DISCOVERY CONFIG CHANGED')
        
    # UPDATE BACKLOG
    raw_backlog = event['args']['backlog']
    backlog = filter_backlog(raw_backlog)

In [ ]:
def update_middleware():
    
    # PRINT REACTION
    utils.log('MIDDLEWARE UPDATE TRIGGERED')

    # TRIGGER UPDATE SCRIPT
    # subprocess.call('./patcher')

    # CLOSE LANCHER
    # sys.exit(0)

In [ ]:
def perform_task(task):
    
    # SHOW MSG
    utils.log('STARTING TASK: ' + task)

    # PERFORM ORACLE TASK
    result = oracle.perform_task(task)

    # SUBMIT THE TASK RESULT
    task_manager.write({
        'func': 'complete',
        'params': [task, result]
    })

    # SHOW MSG
    utils.log('TASK COMPLETED')

In [ ]:
def process_message(event):

    # SERIALIZE EVENT PARAMS
    author = blockchain.toHex(event['sig'])
    payload = blockchain.toText(event['payload'])

    # DECODE THE PAYLOAD
    data = decode(payload)
    
    # REQUIRED KEYS FOR VALID MESSAGE
    required = ['type', 'discovery']

    # DECODED KEYS
    keys = list(data.keys())
    
    # THE REQUEST KEYWORD FOR THE PAYLOAD TYPE
    keyword = 'request'
    
    # IF THE KEYSETS MATCH & THE TYPE IS A REQUEST
    if (required == keys and data['type'] == keyword):
        
        # CHECK MATCHES IN DISCOVERY PARAMS
        discovery_result = compare_discovery(data['discovery'], discovery_config)
        
        # IF EVERYTHING MATCHED
        if (discovery_result.count(False) == 0):
            
            # SHOW MSG
            utils.log('DISCOVERY REQUEST DETECTED')
            
            # ENCODE A JSON RESPONSE
            response = encode({
                'type': 'response',
                'source': payload,
                'device': device.hash
            })
            
            # SLEEP FOR 2 SECONDS
            time.sleep(2)

            # RESPOND TO REQUEST
            shh.post({
                'symKeyID': settings['whisper']['symkey'],
                'payload': blockchain.toHex(text=response),
                'topic': blockchain.toHex(text=settings['whisper']['topic']),
                'sig': whisper_id,
                'powTarget': 2.5,
                'powTime': 2
            })

### CONTRACT EVENTS

In [ ]:
update_event = oracle.event('middleware')

In [ ]:
modification_event = oracle.event('modification')

In [ ]:
message_event = shh.newMessageFilter({
    'topic': web3.toHex(text=settings['whisper']['topic']),
    'symKeyID': settings['whisper']['symkey']
})

### EVENT LOOP

In [ ]:
try:
    utils.log('AWAITING EVENTS...\n')
    
    while(True):
    
        # FETCH THE GLOBAL PARAMS
        global backlog
        global active
        global discoverable

        # VARIABLE MODIFICATION EVENT
        for event in modification_event.get_new_entries():
            update_details(event)

        # UPDATE MIDDLEWARE EVENT
        for event in update_event.get_new_entries():
            update_middleware()

        # IF THE DEVICE IS SET TO ACTIVE
        if (active):

            # PERFORM TASKS IN BACKLOG
            for task in backlog:
                perform_task(task)
        
        # IF THE DEVICE IS DISCOVERABLE
        if (discoverable):
            
            # TRACK WHISPER REQUESTS
            for event in shh.getMessages(message_event):
                process_message(event)
        
# KILL PROCESS
except:
    utils.log('THE PROCESS WAS MANUALLY KILLED!')